In [2]:
from nltk import word_tokenize, pos_tag    
         
pos_all = ['inspiring', 'exotic', 'good-looking', 'effective', 'gripping', 'thrilling', 'intriguing', 'satisfying', 'entertaining', 'stylish', 
        'funny', 'emotional', 'naturalistic', 'romantic', 'resonant', 'brilliant', 'absorbing', 'fresh', 'lyrical', 'honest', 'clever']
neg_all = ['silly', 'sour', 'cynical', 'amateurish', 'offensive', 'stupid', 'dishonest', 'r-rated', 'rough', 'unsuccessful', 
        'unfunny', 'repetitive', 'sappy', 'dull', 'dry', 'mush-hearted', 'predictable', 'creepy', 'neurotic', 'disturbing']

negation_set = ('no', 'not', "isn't", "wasn't", "weren't", 'never', 'seldom', 'barely', 'rarely', 'hardly', 'overly', 'excessively', 'too')

# load corpus & tokenize by sentence & POS_tag
corpus = []
f = open('corpus2/rt-polarity.neg', "r", encoding='latin-1')
raw = f.read().split('\n')
for rev in raw:
    corpus.append(pos_tag(word_tokenize(rev)))

f = open('corpus2/rt-polarity.pos', "r", encoding='latin-1')
raw = f.read().split('\n')
for rev in raw:
    corpus.append(pos_tag(word_tokenize(rev)))

In [3]:
print('corpus:')
for corpora in corpus:
    print(corpora)

corpus:
[('simplistic', 'JJ'), (',', ','), ('silly', 'RB'), ('and', 'CC'), ('tedious', 'JJ'), ('.', '.')]
[('it', 'PRP'), ("'s", 'VBZ'), ('so', 'RB'), ('laddish', 'JJ'), ('and', 'CC'), ('juvenile', 'NN'), (',', ','), ('only', 'RB'), ('teenage', 'NN'), ('boys', 'NNS'), ('could', 'MD'), ('possibly', 'RB'), ('find', 'VB'), ('it', 'PRP'), ('funny', 'JJ'), ('.', '.')]
[('exploitative', 'JJ'), ('and', 'CC'), ('largely', 'RB'), ('devoid', 'NN'), ('of', 'IN'), ('the', 'DT'), ('depth', 'NN'), ('or', 'CC'), ('sophistication', 'NN'), ('that', 'WDT'), ('would', 'MD'), ('make', 'VB'), ('watching', 'NN'), ('such', 'PDT'), ('a', 'DT'), ('graphic', 'JJ'), ('treatment', 'NN'), ('of', 'IN'), ('the', 'DT'), ('crimes', 'NNS'), ('bearable', 'JJ'), ('.', '.')]
[('[', 'NN'), ('garbus', 'NN'), (']', 'NN'), ('discards', 'VBZ'), ('the', 'DT'), ('potential', 'NN'), ('for', 'IN'), ('pathological', 'JJ'), ('study', 'NN'), (',', ','), ('exhuming', 'VBG'), ('instead', 'RB'), (',', ','), ('the', 'DT'), ('skewed', 'JJ

In [4]:
pos_seed = pos_all
neg_seed = neg_all
delete_buffer = []
epoch = 0
add_dict = {}
while True:
    # print(f'in epoch {epoch}')
    # print('pos seed:')
    # print(pos_seed)
    # print('neg seed:')
    # print(neg_seed)
    pos_add = []
    neg_add = []
    # pick new adj
    stable = True
    for corpora in corpus:
        for i in range(len(corpora)-1):
            # same polarity
            if corpora[i][1] == 'JJ' and corpora[i][0] in pos_seed:
                if corpora[i+1][0] in ['and', ',', '&']:
                    # pattern: pos and JJ -> pol(JJ) = +
                    if corpora[i+2][1] == 'JJ':
                        stable = False
                        if corpora[i+2][0] in add_dict:
                            add_dict[corpora[i+2][0]] += 1
                        else:
                            add_dict[corpora[i+2][0]] = 1
                            pos_add.append(corpora[i+2][0])
                    # pattern: pos and not JJ -> pol(JJ) = -
                    elif corpora[i+2][0] in negation_set and corpora[i+3][1] == 'JJ':
                        stable = False
                        if corpora[i+3][0] in add_dict:
                            add_dict[corpora[i+3][0]] -= 1
                        else:
                            add_dict[corpora[i+3][0]] = -1
                            neg_add.append(corpora[i+3][0])
                elif corpora[i+1][0] == 'but':
                    # pattern: pos but JJ -> pol(JJ) = -
                    if corpora[i+2][1] == 'JJ':
                        stable = False
                        if corpora[i+2][0] in add_dict:
                            add_dict[corpora[i+2][0]] -= 1
                        else:
                            add_dict[corpora[i+2][0]] = -1
                            neg_add.append(corpora[i+2][0])
                    # pattern: pos but not JJ -> pol(JJ) = +
                    elif corpora[i+2][0] in negation_set and corpora[i+3][1] == 'JJ':
                        stable = False
                        if corpora[i+3][0] in add_dict:
                            add_dict[corpora[i+3][0]] += 1
                        else:
                            add_dict[corpora[i+3][0]] = 1
                            pos_add.append(corpora[i+3][0])
            if corpora[i][1] == 'JJ' and corpora[i][0] in neg_seed:
                if corpora[i+1][0] in ['and', ',', '&']:
                    # pattern: neg and JJ -> pol(JJ) = -
                    if corpora[i+2][1] == 'JJ':
                        stable = False
                        if corpora[i+2][0] in add_dict:
                            add_dict[corpora[i+2][0]] -= 1
                        else:
                            add_dict[corpora[i+2][0]] = -1
                            neg_add.append(corpora[i+2][0])
                    # pattern: neg and not JJ -> pol(JJ) = +
                    elif corpora[i+2][0] in negation_set and corpora[i+3][1] == 'JJ':
                        stable = False
                        if corpora[i+3][0] in add_dict:
                            add_dict[corpora[i+3][0]] += 1
                        else:
                            add_dict[corpora[i+3][0]] = 1
                            pos_add.append(corpora[i+3][0])
                elif corpora[i+1][0] == 'but':
                    # pattern: neg but JJ -> pol(JJ) = +
                    if corpora[i+2][1] == 'JJ':
                        stable = False
                        if corpora[i+2][0] in add_dict:
                            add_dict[corpora[i+2][0]] += 1
                        else:
                            add_dict[corpora[i+2][0]] = 1
                            pos_add.append(corpora[i+2][0])
                    # pattern: neg but not JJ -> pol(JJ) = -
                    elif corpora[i+2][0] in negation_set and corpora[i+3][1] == 'JJ':
                        stable = False
                        if corpora[i+3][0] in add_dict:
                            add_dict[corpora[i+3][0]] -= 1
                        else:
                            add_dict[corpora[i+3][0]] = -1
                            neg_add.append(corpora[i+3][0])

    if len(pos_add) == 0 and len(neg_add) == 0:
        break
    # add new seeds to repo and prepare seeds for next iteration
    # resuse delete buffer to avoid create extras
    pos_add = list(dict.fromkeys(pos_add))
    delete_buffer.clear()
    for seed in pos_add:
        if seed in pos_all:
            delete_buffer.append(seed)
    for seed in delete_buffer:
        pos_add.remove(seed)
    neg_add = list(dict.fromkeys(neg_add))
    delete_buffer.clear()
    for seed in neg_add:
        if seed not in neg_all:
            delete_buffer.append(seed)
    for seed in delete_buffer:
        neg_add.remove(seed)
    pos_seed = pos_add
    neg_seed = neg_add
    epoch += 1

In [5]:
pos_add = []
neg_add = []
# print(add_dict)
for key, value in add_dict.items():
    if value > 0 and key not in pos_all:
        pos_add.append(key)
    elif value < 0 and key not in neg_all:
        neg_add.append(key)
print('final pos list:')
print(pos_add)
print('final neg list:')
print(neg_add)

final pos list:
['inoffensive', 'vital', 'concerned', 'full', 'nasty', 'tragic', 'light', 'placid', 'claustrophobic', 'good-natured', 'delightful', 'aggressive', 'free', 'fast', 'rapturous', 'lightweight', 'unselfconscious', 'engaging', 'energetic', 'quirky', 'unforced', 'uplifting', 'amusing', 'touching', 'informative', 'honorable', 'subtle', 'twisted', 'poignant', 'observant', 'brutal', 'human', 'well-contructed', 'raw', 'rocky-like', 'thought-provoking', 'self-aggrandizing', 'pithy', 'sexy', 'moral', 'suggestive', 'harmless', 'sad', 'open-hearted', 'semi-surrealist', 'deeply', 'richly', 'weird', 'scary', 'smart', 'albeit', 'cerebral', 'celebratory', 'old-fashioned', 'insightful', 'action-packed', 'sugary', 'well-crafted', 'interesting', 'tuck', 'innocuous', 'rote', 'unmemorable', 'cheerful', 'sassy', 'sick', 'fuddy-duddy', 'sordid', 'soggy', 'extreme-sports', 'superior', 'oscar-nominated', 'powerful', 'alive', 'rapid-fire', 'unpleasant', 'spiritual', 'such', 'wacky', 'near-masterpie

In [6]:
correct_pos = ['inoffensive', 'vital', 'concerned', 'full', 'placid', 'good-natured', 'delightful', 'free', 
               'rapturous', 'unselfconscious', 'engaging', 'energetic', 'unforced', 'uplifting', 'amusing', 
               'touching', 'informative', 'honorable', 'subtle', 'poignant', 'observant', 'human', 
               'well-contructed', 'thought-provoking', 'pithy', 'sexy', 'moral', 'suggestive', 'harmless', 
               'open-hearted','semi-surrealist', 'scary', 'smart', 'celebratory', 'insightful', 
               'action-packed', 'sugary', 'well-crafted', 'interesting', 'innocuous', 'cheerful', 
               'superior', 'oscar-nominated', 'powerful', 'alive', 'spiritual', 'wacky', 'near-masterpiece', 
               'empathetic', 'fun', 'cinemantic', 'compulsive', 'original', 'reflective', 'witty', 
               'often-funny', 'solid', 'wonderful', 'violent', 'visceral', 'savvy', 'likable', 
               'sophisticated', 'affecting', 'reasonable', 'inventive', 'accessible', 'seductive', 
               'refined', 'sobering', 'universal', 'well-formed', 'admirable', 'kinetically-charged', 
               'imaginative', 'low-key',  'inspired', 'hilarious', 'unassuming', 'enjoyable', 'plot', 
               'modern', 'organic', 'philosophical', 'impressive', 'horrid', 'artful', 'well-shot', 
               'straightforward', 'ambitious', 'frankly', 'hard-edged', 'prescient', 'heart-felt', 'rapt', 
               'unsentimental', 'unusual', 'poetic', 'literate', 'high-adrenaline', 'surprising', 
               'successful', 'rich', 'personal', 'sudden', 'eager', 'well-intentioned', 'easy', 'suspenseful', 'intelligent', 'dramatic', 
                'food-for-thought', 'exciting', 'bittersweet', 'luscious', 'thoughtful', 'well-acted', 
               'truthful', 'calculated', 'loud', 'reverent', 'comfortable', 'obvious', 'unflinching', 
               'low-budget', 'bang-the-drum', 'lengthy', 'character-driven', 'objective', 'well-directed', 
               'complex', 'thoroughly', 'stiff', 'didactic', 'important', 'generous', 'unsatisfied', 
               'ponderous', 'predictable', 'exhilarating', 'meditative', 'meaningful', 'uncompromising', 
               'clinical']

correct_neg = ['infantile', 'pointless', 'irrelevant', 'small', 'unentertaining', 'pretentious', 
               'unrecommendable', 'self-indulgent', 'muddy', 'lazy', 'over-the-top', 'brain-deadening', 
               'quasi-improvised', 'unoriginal', 'spiritless', 'redundant', 'bloody', 'wooden', 
               'too-tepid', 'outrageous']

print(f'# of new positives: {len(pos_add)}; # of correct positives: {len(correct_pos)}; accuracy: {len(correct_pos)/len(pos_add)}')
print(f'# of new negatives: {len(neg_add)}; # of correct negatives: {len(correct_neg)}; accuracy: {len(correct_neg)/len(neg_add)}')
print(f'overall accuracy: {( len(correct_pos)+len(correct_neg)) / (len(pos_add)+len(neg_add) )}')

# of new positives: 256; # of correct positives: 144; accuracy: 0.5625
# of new negatives: 34; # of correct negatives: 20; accuracy: 0.5882352941176471
overall accuracy: 0.5655172413793104
